Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Hung-Wei Chang"
COLLABORATORS = ""

---

<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Assignment 5

In [2]:
import pandas as pd
import sqlite3 as db
import numpy as np

In [3]:
# Run this cell before any others below
# Download the data files into the same directory where you have saved this notebook

import urllib.request
from pathlib import Path
import os

db_source = 'https://storage.googleapis.com/aipi_datasets/flights.db'
db_destination = str(Path()) + '/flights.db'

# If the file does not already exist in the directory, download it
if not os.path.exists(db_destination):
        urllib.request.urlretrieve(db_source,db_destination)

In [4]:
# Connect to the database
conn = db.connect('flights.db')

# Create a 'cursor' for executing commands
c = conn.cursor()

In [5]:
# Rename tables for clarity

c.execute('ALTER TABLE flight_data RENAME TO Flights')
c.execute('ALTER TABLE airport_codes RENAME TO Airports')
c.execute('ALTER TABLE airline_codes RENAME TO Airlines')

# Rename columns to conform to best practices for primary key. Note that Airline_ID key is not an integer, 
# which is non-standard but is permitted for primary keys 

c.execute('ALTER TABLE Flights RENAME COLUMN OP_UNIQUE_CARRIER TO Airline_ID')
c.execute('ALTER TABLE Airports RENAME COLUMN Code TO Airport_ID')
c.execute('ALTER TABLE Airlines RENAME COLUMN Code TO Airline_ID')

In [6]:
# Let's look at all the data in each table (generally you don't want to do this if the tables are very large)
flights_df = pd.read_sql_query("SELECT * FROM Flights", conn)
airports_df = pd.read_sql_query("SELECT * FROM Airports", conn)
airlines_df = pd.read_sql_query("SELECT * FROM Airlines", conn)
display(flights_df.head())
display(airports_df.head())
display(airlines_df.head())


,DAY_OF_WEEK,FL_DATE,Airline_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_DELAY_NEW,CANCELLED,WEATHER_DELAY
0,1,8/12/19,9E,5001,10397,10781,1050,0.0,0,NaN
1,1,8/19/19,9E,5001,10397,10781,1050,0.0,0,NaN
2,1,8/26/19,9E,5001,10397,10781,1050,0.0,0,NaN
3,1,8/5/19,9E,5001,10781,10397,1159,74.0,0,0.0
4,1,8/12/19,9E,5001,10781,10397,1159,0.0,0,NaN


,Airport_ID,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


,Airline_ID,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


### Question 1 [10 pts]
Complete the below function `get_airport_description()` which receives an Airport_ID as input and returns the airport's Description from the Airports table (exactly as stated in the Description field) **as a string**.  Your function must perform a SQL query (you cannot simply extract the Description from the above DataFrame).

In [197]:
def get_airport_description(airport_id,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect(db_name)
    c = conn.cursor()
    query = '''
            SELECT Airports.Airport_ID  , Airports.Description
            FROM Airports
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    desc = str(dict(results)[int(airport_id)])
    return desc
    raise NotImplementedError()

In [198]:
# Test cell for get_airport_description
airport = get_airport_description(11995,'flights.db')
assert airport=='Greensboro/High Point, NC: Piedmont Triad International'


### Question 2 [10 pts]
Complete the below function `get_airline_id()` which takes as input an airline's Description as a string (e.g. 'Titan Airways') and returns the airline's Airline_ID **as a string**.  Your function must perform a SQL query to get the answer.

In [195]:
def get_airline_id(airline_description,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect(db_name)
    c = conn.cursor()
    query = '''
            SELECT Airlines.Description, Airlines.Airline_ID
            FROM Airlines
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    airline_id = str(dict(results)[airline_description])
    return airline_id
    
    raise NotImplementedError()


In [196]:
# Test cell for get_airline_id()
print('Your function returned the Airline_ID for Skytrain as: {}'.format(get_airline_id('Skytrain','flights.db')))


Your function returned the Airline_ID for Skytrain as: SKT


### Question 3 [10 pts]
Complete the below function `get_airport_destinations_for_airline()` which takes as input a string Airline_ID and returns a **list of the unique airport ids (DEST_AIRPORT_ID) as integers** for all the airports to which the the airline flew at least once (with no duplicates). Your function must perform a SQL query to get the answer.


In [193]:
def get_airport_destinations_for_airline(airline_id,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.Airline_ID, Flights.DEST_AIRPORT_ID
            FROM Flights
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    new_list = [int(x[1]) for x in results if (x[0] == airline_id) ]
    seen = set()
    seen_add = seen.add
    final_list=  [int(x) for x in new_list if not (x in seen or seen_add(x))]
    return final_list
    raise NotImplementedError()

In [194]:
# Test cell
airports = get_airport_destinations_for_airline('9E','flights.db')
assert airports == [10781, 10397, 14098, 13931, 12478, 10792, 11721, 12953, 10868, 11057, 11193, 
                    10185, 11042, 11433, 11003, 10581, 15412, 11977, 11278, 10821, 10990, 10721, 
                    12266, 13487, 11612, 14814, 10431, 11953, 11617, 10529, 10980, 12197, 12339, 
                    14524, 12945, 14492, 12884, 10685, 15249, 11150, 10785, 15919, 13244, 11973, 
                    11986, 10208, 14122, 14685, 10408, 11267, 13930, 14783, 12951, 14321, 11298, 
                    14100, 11996, 13198, 11066, 14576, 11423, 13795, 13367, 10693, 11995, 13184, 
                    12264, 13422, 13485, 15096, 13230, 14683, 14574, 13342, 11203, 12451, 15370, 
                    15016, 10257, 11823, 11618, 10874, 10135, 13495, 13029, 14633, 13433, 10423, 
                    13577, 13296, 15380, 13541, 10154, 10994, 15304, 13303, 14457, 14027, 13204, 
                    15323, 10599]


### Question 4 [10 pts]
Complete the below function `get_airport_descriptions_for_airline()` which takes as input an airline's string Description (in the format of the Description field) and returns a **list of the unique airport Descriptions as strings** (the Description field in the Airports table) to which the airline flew one or more flights (with no duplicates).  Your function must perform a SQL query to get the answer.

In [13]:
def get_airport_descriptions_for_airline(airline_description,db_name='flights.db'):
    # YOUR CODE HERE
    airline_id = get_airline_id(airline_description,db_name='flights.db')
    airports_id = get_airport_destinations_for_airline(airline_id,'flights.db')
    airport_description_list = [get_airport_description(x, db_name='flights.db') for x in airports_id ]
    return airport_description_list
    
#     conn = db.connect('flights.db')
#     c = conn.cursor()
#     query = '''
#             SELECT Airports.Airport_ID, Airports.Description
#             FROM Airports 

#             '''
#     c.execute(query)
#     results = c.fetchall()
#     c.close()
#     conn.close()
#     results
#     unique = []
#     for tup in results:
#         if tup not in unique:
#             unique.append(tup)
#     desc_dict = dict(unique)
#     airline_id = get_airline_id(airline_description,db_name='flights.db')
#     airports_id_lis = get_airport_destinations_for_airline(airline_id,'flights.db')
#     final = [desc_dict[x] for x in airports_id_lis]
#     return final
    raise NotImplementedError()

In [121]:
# Test cell
dests = get_airport_descriptions_for_airline('Republic Airline','flights.db')
print('Republic Airline flew to {} destination airports'.format(len(dests)))
print(dests)


Republic Airline flew to 97 destination airports
['Houston, TX: George Bush Intercontinental/Houston', 'Washington, DC: Ronald Reagan Washington National', 'Pittsburgh, PA: Pittsburgh International', 'Philadelphia, PA: Philadelphia International', 'New Orleans, LA: Louis Armstrong New Orleans International', 'Miami, FL: Miami International', 'Kansas City, MO: Kansas City International', 'Raleigh/Durham, NC: Raleigh-Durham International', 'New York, NY: LaGuardia', 'Nashville, TN: Nashville International', 'Louisville, KY: Louisville Muhammad Ali International', 'Richmond, VA: Richmond International', 'Boston, MA: Logan International', 'Buffalo, NY: Buffalo Niagara International', 'Sarasota/Bradenton, FL: Sarasota/Bradenton International', 'Indianapolis, IN: Indianapolis International', 'Columbus, OH: John Glenn Columbus International', 'St. Louis, MO: St Louis Lambert International', 'New York, NY: John F. Kennedy International', 'Detroit, MI: Detroit Metro Wayne County', 'Lexington, K

### Question 5 [10 pts]
Complete the below function `get_airlines_with_no_flights()` which takes as input the name of the database and returns a **list of tuples** each containing the string Airline_ID and string airline Description (from the Airlines table) of the unique set of airlines (no duplicates) which had no flights (e.g. ('02Q', 'Titan Airways')).  Your function must perform a SQL query to get the answer.

In [151]:
 def get_airlines_with_no_flights(db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect(db_name)
    c = conn.cursor()
    query = '''
            SELECT Airlines.Airline_ID  , Airlines.Description
            FROM Airlines INNER JOIN Flights on Airlines.Airline_ID = Flights.Airline_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()

    seen = set()
    seen_add = seen.add
    check_list=  [x for x in results if not (x in seen or seen_add(x))]

    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Airlines.Airline_ID , Airlines.Description
            FROM Airlines 
            '''
    c.execute(query)
    results_2 = c.fetchall()
    c.close()
    conn.close()
    final_list = [ x for x in results_2 if x not in results]
    return final_list
    
    raise NotImplementedError()

In [153]:
# Test cell
results = get_airlines_with_no_flights('flights.db')
print('Your function found {} airlines with no flights'.format(len(results)))
print(results)


Your function found 1656 airlines with no flights
[('02Q', 'Titan Airways'), ('04Q', 'Tradewind Aviation'), ('05Q', 'Comlux Aviation, AG'), ('06Q', 'Master Top Linhas Aereas Ltd.'), ('07Q', 'Flair Airlines Ltd.'), ('09Q', 'Swift Air, LLC d/b/a Eastern Air Lines d/b/a Eastern'), ('0BQ', 'DCA'), ('0CQ', 'ACM AIR CHARTER GmbH'), ('0FQ', 'Maine Aviation Aircraft Charter, LLC'), ('0GQ', 'Inter Island Airways, d/b/a Inter Island Air'), ('0HQ', 'Polar Airlines de Mexico d/b/a Nova Air'), ('0J', 'JetClub AG'), ('0JQ', 'Vision Airlines'), ('0LQ', 'Metropix UK, LLP.'), ('0OQ', 'Open Skies'), ('0Q', 'Flying Service N.V.'), ('0QQ', 'TAG Aviation (UK) Ltd.'), ('0RQ', 'TAG Aviation Espana S.L.'), ('0TQ', 'Corporatejets, XXI'), ('0UQ', 'Comlux Malta, Ltd.'), ('0VQ', 'Ocean Sky (UK) Limited'), ('0WQ', 'Avjet Corporation'), ('0YQ', 'Comlux Malta Ltd.'), ('10Q', 'Swiss Air Ambulance'), ('12Q', 'Unijet'), ('13Q', 'Chartright Air Inc.'), ('14Q', 'London Air Services Limited'), ('15Q', 'Air Alsie A/S'), ('

### Question 6 [10 pts]
Complete the below function `get_n_flights_for_airport_id()` which takes an integer input Airport_ID and returns a tuple containing the airport's Description (as a string) and the count of how many flights landed there (as an integer).  Your function must perform a SQL query to get the answer.

In [191]:
def get_n_flights_for_airport_id(airport_id,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.DEST_AIRPORT_ID  ,  Airports.Description, COUNT(Flights.DEST_AIRPORT_ID)
            FROM Flights Left Join Airports on Flights.DEST_AIRPORT_ID = Airports.Airport_ID
            GROUP BY Flights.DEST_AIRPORT_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    dic = dict([( x[0] , (x[1], int(x[2])) ) for x in results])
    return dic[airport_id]
    raise NotImplementedError()

In [192]:
# Test cell
flight_cnt = get_n_flights_for_airport_id(10781)
print(f'For the airport {flight_cnt[0]} there were {flight_cnt[1]} flights')


For the airport Baton Rouge, LA: Baton Rouge Metropolitan/Ryan Field there were 635 flights


### Question 7 [10 pts]
Complete the below function `get_airports_with_max_flights()` which returns a list of tuples containing the Airport_ID (as an integer), Description (as a string) and number of flights (as an integer) for the 10 airports which had the most flights landing there.  Your function must perform a SQL query to get the answer.

In [74]:
def get_airports_with_max_flights(db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.DEST_AIRPORT_ID  ,  Airports.Description, COUNT(Flights.DEST_AIRPORT_ID)
            FROM Flights Left Join Airports on Flights.DEST_AIRPORT_ID = Airports.Airport_ID
            GROUP BY Flights.DEST_AIRPORT_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()

    df = pd.DataFrame(results, columns = ['DEST_AIRPORT_ID', 'Airports.Description', 'COUNT'])

    df = df.sort_values(by=['COUNT'], ascending = False )
    df[['DEST_AIRPORT_ID', 'COUNT']].astype(int)

    records = df.iloc[:10].to_records(index=False)
    final = list(records)
    return final
    
    raise NotImplementedError()

In [75]:
# Test cell
airports = get_airports_with_max_flights('flights.db')
print(airports)


[(10397, 'Atlanta, GA: Hartsfield-Jackson Atlanta International', 35434), (13930, "Chicago, IL: Chicago O'Hare International", 30770), (11298, 'Dallas/Fort Worth, TX: Dallas/Fort Worth International', 27827), (11292, 'Denver, CO: Denver International', 23302), (11057, 'Charlotte, NC: Charlotte Douglas International', 20317), (12892, 'Los Angeles, CA: Los Angeles International', 19818), (14771, 'San Francisco, CA: San Francisco International', 16721), (12953, 'New York, NY: LaGuardia', 15269), (13487, 'Minneapolis, MN: Minneapolis-St Paul International', 15200), (12266, 'Houston, TX: George Bush Intercontinental/Houston', 15200)]


### Question 8 [10 pts]
Complete the function `get_average_delay()` which takes as input a string Airline_ID and returns the total average departure delay (DEP_DELAY_NEW) for the airport as a float.  Your function must perform a SQL query to get the answer.

In [81]:
def get_average_delay(airline_id,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.Airline_ID  , AVG(Flights.DEP_DELAY_NEW)
            FROM Flights Left Join Airlines on Flights.Airline_ID = Airlines.Airline_ID
            GROUP BY Airlines.Airline_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()
    return dict(results)[airline_id]
    raise NotImplementedError()

In [82]:
# Test cell
delay = get_average_delay('9E','flights.db')
print('Your function calculated the average delay for airline with id 9E as {:.3f}'.format(delay))


Your function calculated the average delay for airline with id 9E as 15.097


### Question 9 [10 pts]
Complete the below function `get_longest_delay()` which returns a tuple containing the Airline_ID (as a string), the airline's Description (as a string), the weather delay (WEATHER_DELAY) as a float, the origin airport string Description, and the destination airport string Description for the flight with the longest weather delay (in the order specified). Your function must perform a SQL query to get the answer.

In [189]:
def get_longest_delay(db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.Airline_ID, air.Description, max(Flights.WEATHER_DELAY), ori.Description, des.Description 
            FROM Flights Left Join Airlines air on Flights.Airline_ID = air.Airline_ID
            Left Join Airports ori on Flights.ORIGIN_AIRPORT_ID = ori.Airport_ID
            Left Join Airports des on Flights.DEST_AIRPORT_ID = des.Airport_ID
            GROUP BY  Flights.Airline_ID, Flights.ORIGIN_AIRPORT_ID, Flights.DEST_AIRPORT_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()

    results
    df = pd.DataFrame(results, columns = ['airline_id', 'airline_des', 'weather_delay', 'ori_airport', 'des_airport'])
    df['weather_delay'] = df['weather_delay'].fillna(0)
    df['weather_delay'].astype(float)
    df[ ['airline_id', 'airline_des','ori_airport', 'des_airport'] ].astype(str)
    df_final = df.iloc[df['weather_delay'].idxmax()]
    final = tuple(df_final.to_list()) 

    return final

    raise NotImplementedError()

In [190]:
# Test cell
longest_delay = get_longest_delay('flights.db')
print(longest_delay)


('OO', 'SkyWest Airlines Inc.', 1440.0, 'Hartford, CT: Bradley International', "Chicago, IL: Chicago O'Hare International")


### Question 10 [10 pts]
Complete the below function `get_n_airports_serviced()` which takes a string Airline_ID as input and returns the number of airports the airline departs from, as an integer.  Your function must perform a SQL query to get the answer.

In [100]:
def get_n_airports_serviced(airline_id,db_name='flights.db'):
    # YOUR CODE HERE
    conn = db.connect('flights.db')
    c = conn.cursor()
    query = '''
            SELECT Flights.Airline_ID , Flights.ORIGIN_AIRPORT_ID, Count(Flights.ORIGIN_AIRPORT_ID)
            FROM Flights Left Join Airlines on Flights.Airline_ID = Airlines.Airline_ID
            GROUP BY Airlines.Airline_ID, Flights.ORIGIN_AIRPORT_ID
            '''
    c.execute(query)
    results = c.fetchall()
    c.close()
    conn.close()

    lis =[x[0] for x in results]
    num = lis.count(airline_id)
    return num 
    raise NotImplementedError()

In [101]:
# Test cell
count = get_n_airports_serviced('9E','flights.db')
print(f'Your function found that airline with Airline_ID 9E departs from {count} airports')


Your function found that airline with Airline_ID 9E departs from 101 airports
